In [ ]:
# Install Pytorch & other libraries
!pip install "torch==2.1.2" tensorboard

# Install Hugging Face libraries
!pip install  --upgrade \
  "transformers==4.36.2" \
  "datasets==2.16.1" \
  "accelerate==0.26.1" \
  "evaluate==0.4.1" \
  "bitsandbytes==0.42.0" \
  "trl==0.7.10"  \
  "peft==0.7.1" \

# install peft & trl from github
!pip install git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e --upgrade
!pip install git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f --upgrade

In [ ]:
model_path = "path_of_backwards_model"
data_path = "path_of_conversations_cureated_in_previous_notebook"

In [ ]:

from huggingface_hub import login
import os
 
login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=False
)
 

## Load the Fine-Tuned Model for Inference

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
 
peft_model_id = model_path
# peft_model_id = args.output_dir
 
# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

## Run Inference on test samples

In [ ]:
from datasets import load_dataset
from random import randint

 
# Load our test dataset
eval_dataset = load_dataset("json", data_files=data_path,  split="train")

In [ ]:
def eval_entry(idx:int, max_tokens = 100):
    prompt = pipe.tokenizer.apply_chat_template(
        eval_dataset[idx]["data"][:2], 
        tokenize=False, add_generation_prompt=True)
    
    outputs = pipe(prompt, 
                   max_new_tokens=max_tokens, 
                   do_sample=False, temperature=0.1, 
                   top_k=50, top_p=0.1, 
                   eos_token_id=pipe.tokenizer.eos_token_id, 
                   pad_token_id=pipe.tokenizer.pad_token_id)
    
    length = len(prompt)
    content = outputs[0]['generated_text'][length:].strip()
    del prompt
    del outputs
    
    # Clear GPU cache
    torch.cuda.empty_cache()

    # Collect garbage
    gc.collect()
    
    return content

In [ ]:
generated_responses = []

for i in range(0, len(eval_dataset)):
    generated_responses.append(eval_entry(i))
    # Clear GPU cache
    torch.cuda.empty_cache()

    # Collect garbage
    gc.collect()